In [2]:
import pandas as pd
import numpy as np
import json # Para trabajar con JSON
from pandas import json_normalize
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
#import nltk
#from wordcloud import WordCloud
#from textblob import TextBlob
#from nltk.corpus import stopwords
#from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Descargar los stopwords en español e inglés
#nltk.download('stopwords')
#stop_words = set(stopwords.words('english') + stopwords.words('spanish'))

import warnings
warnings.filterwarnings("ignore")

Extraemos y transformamos los df de los estados seleccionados para el trabajo: Florida, California, New York, Texas, Ilinois y Arizona.

In [3]:
df_florida_1 = pd.read_json("C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Florida/1.json", lines=True)

In [4]:
df_florida_1.describe()

,user_id,time,rating
count,1.500000e+05,1.500000e+05,150000.000000
mean,1.095940e+20,1.557095e+12,4.334360
std,5.151720e+18,4.997719e+10,1.280953
min,1.000002e+20,1.052611e+12,1.000000
25%,1.054386e+20,1.524675e+12,4.000000
50%,1.095449e+20,1.561863e+12,5.000000
75%,1.139628e+20,1.598543e+12,5.000000
max,1.184467e+20,1.631205e+12,5.000000


In [5]:
df_florida_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     100023 non-null  object 
 5   pics     4577 non-null    object 
 6   resp     31967 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB


In [6]:
df_florida_1.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


El df muestra usuarios con su nombre, id y el rating (int 1 a 5) que ha dado a locales, mas una reseña en texto, seguido de una foto y la respuesta del negocio, por ultimo el id de google map.

In [7]:
display(df_florida_1.isnull().sum())

user_id         0
name            0
time            0
rating          0
text        49977
pics       145423
resp       118033
gmap_id         0
dtype: int64

Luego de una primer analisis observamos que hay muchos nulos en las columnas text, pics y resp. Tambien entendemos que, para cumplir con los requisitos del cliente y entrenar el modelo del sistema de recomendacion las columnas time, pics y resp no son necesarias. Por consiguiente primero se eliminaran las columnas pics y resp y se volveran a analizar los valores nulos 

In [8]:
# eliminar columnas 'pics' y 'resp'
df_florida_1 = df_florida_1.drop(columns=['time','pics','resp'])

In [9]:
df_florida_1.columns

Index(['user_id', 'name', 'rating', 'text', 'gmap_id'], dtype='object')

In [10]:
# Conteo de valores nulos en cada columna
print("Valores nulos en cada columna:")
display(df_florida_1.isnull().sum())

Valores nulos en cada columna:


user_id        0
name           0
rating         0
text       49977
gmap_id        0
dtype: int64

Los valores nulos representan reseñas valordas con estrellas, pero no asi con texto. Como los comenarios escritos son utiles para entrenar el modelo del sistema de recomendacion y las estrellas tambien, no se eliminaran los valores nulos en la columna 'text', se los reemplazara con la leyenda "sin comentario"

In [11]:
# Reemplazar los valores nulos en la columna 'text'  
df_florida_1['text'] = df_florida_1['text'].fillna('sin comentario')

Entendiendo que los archivos dentro de la carpeta pertenecen al mismo estado de Arizona, se podria decir que son el mismo df de lestado dividido en 14 partes.

---

Por otro lado y luego de un analisis rapido encontramos que los siguientes data sets comparten las mismas caracteristicas por lo que se desarrolla una funsion para que haga la transformacion de forma automatica.

ETL Reviews-Florida

In [3]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Florida/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_Florida = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_Florida.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Florida/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     100023 non-null  object 
 5   pics     4577 non-null    object 
 6   resp     31967 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        49977
pics       145423
resp       118033
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Florida/10.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 

In [4]:
df_Florida.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Florida/df_Florida.parquet') 

---

ETL Reviews-California

In [5]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-California/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_California = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_California.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-California/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     89135 non-null   object 
 5   pics     5632 non-null    object 
 6   resp     16935 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        60865
pics       144368
resp       133065
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-California/10.json
<class 'pandas.core.frame.DataFrame'>
RangeI

In [6]:
df_California.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-California/df_California.parquet') 

---

ETL Reviews-New_York

In [18]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-New_York/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_New_York = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_New_York.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-New_York/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     87324 non-null   object 
 5   pics     4596 non-null    object 
 6   resp     16838 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        62676
pics       145404
resp       133162
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-New_York/10.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex

In [19]:
df_New_York.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-New_York/df_New_York.parquet')

---

ETL Reviews-Texas

In [20]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Texas/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_Texas = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_Texas.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Texas/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     95777 non-null   object 
 5   pics     4631 non-null    object 
 6   resp     28224 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        54223
pics       145369
resp       121776
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Texas/10.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500

In [21]:
df_Texas.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Texas/df_Texas.parquet')

---

ETL Reviews-Illinois

In [23]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Illinois/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_Illinois = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_Illinois.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Illinois/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     86973 non-null   object 
 5   pics     3712 non-null    object 
 6   resp     22669 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        63027
pics       146288
resp       127331
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Illinois/10.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex

In [24]:
df_Illinois.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Illinois/df_Illinois.parquet')

---

ETL Reviews-Arizona

In [27]:
import os  

# Definir la función para procesar cada archivo JSON  
def procesar_json(archivo):  
    # Leer el archivo JSON  
    df = pd.read_json(archivo, lines=True)  
    
    # Mostrar información y estadísticas del DataFrame  
    print(f"Procesando: {archivo}")  
    print(df.info())  
    print("Valores nulos en cada columna:")  
    print(df.isnull().sum())  
    
    # Eliminar columnas 'time', 'pics' y 'resp'  
    df = df.drop(columns=['time', 'pics', 'resp'], errors='ignore')  # errors='ignore' para evitar errores si no existen  
    
    # Reemplazar los valores nulos en la columna 'text'  
    df['text'] = df['text'].fillna('sin comentario')  
    
    return df  

# Ruta de la carpeta que contiene los archivos JSON  
ruta_carpeta = "C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Arizona/"  

# Crear una lista para almacenar los DataFrames  
lista_dfs = []  

# Iterar sobre los archivos en la carpeta  
for archivo in os.listdir(ruta_carpeta):  
    if archivo.endswith('.json'):  
        ruta_archivo = os.path.join(ruta_carpeta, archivo)  
        df_procesado = procesar_json(ruta_archivo)  
        lista_dfs.append(df_procesado)  

# Concatenar todos los DataFrames en uno solo  
df_Arizona = pd.concat(lista_dfs, ignore_index=True)  

# Mostrar el DataFrame final  
print(df_Arizona.head()) 

Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Arizona/1.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     99042 non-null   object 
 5   pics     4468 non-null    object 
 6   resp     32403 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB
None
Valores nulos en cada columna:
user_id         0
name            0
time            0
rating          0
text        50958
pics       145532
resp       117597
gmap_id         0
dtype: int64
Procesando: C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Arizona/10.json
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 

In [28]:
df_Arizona.to_parquet('C:/Users/juanv/Downloads/proyecto final/google/reviews-estados/review-Arizona/df_Arizona.parquet')